<a href="https://colab.research.google.com/github/HeroBenHero/HeroColabs/blob/main/Google_Drive_Cloner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Mount Google Drive**
#@markdown Run The Script

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title <-----<strong>Upload your  Rclone.config file</strong>
def moveConfig():
  !mv rclone.conf /root/.config/rclone/rclone.conf

from google.colab import files
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True) 
!curl https://rclone.org/install.sh | sudo bash
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")

In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

In [ ]:
!sudo rmdir /content/Mega
!sudo mkdir /content/Mega
!nohup rclone --vfs-cache-mode writes mount Mega: /content/Mega &

In [ ]:
#@title **Save Location**
#@markdown Copy The Folder Path From Files In Side Menu

Source = "/content/Mega/Classic" #@param {type:"string"}
Destination = "/content/drive/Shareddrives/Shared/GDriveUpload/Classic" #@param {type:"string"}

!sudo cp -av '{Source}' '{Destination}'